In [28]:
import numpy as np
import pandas as pd
import ast
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Embedding, Flatten, Dropout, BatchNormalization, GlobalAveragePooling1D, concatenate
from tensorflow.keras.utils import plot_model
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity

pd.set_option('display.max_columns', None)

data = pd.read_csv("../datasets/sam_df_clean.csv")

# parse genre string as a Python list
def parse_genre(s):
    try: # try to convert string to list
        return ast.literal_eval(s)  
    except (ValueError, SyntaxError):
        # fallback: wrap as a single-element list
        return [str(s)]
data['track_genre'] = data['track_genre'].apply(parse_genre)


# data.head()
print(data.shape)
print(data.columns.tolist())
data.head()

(83679, 20)
['track_id', 'artists', 'album_name', 'track_name', 'popularity', 'duration_ms', 'explicit', 'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'time_signature', 'track_genre']


,track_id,artists,album_name,track_name,popularity,duration_ms,explicit,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,track_genre
0,5SuOikwiRyPMVoIQDJUgSV,Gen Hoshino,Comedy,Comedy,73,230666,False,0.676,0.4610,1,-6.746,0,0.1430,0.0322,0.000001,0.3580,0.715,87.917,4,[acoustic]
1,4qPNDBW1i3p13qLCt0Ki3A,Ben Woodward,Ghost (Acoustic),Ghost - Acoustic,55,149610,False,0.420,0.1660,1,-17.235,1,0.0763,0.9240,0.000006,0.1010,0.267,77.489,4,[acoustic]
2,1iJBSr7s7jYXzM8EGcbK5b,Ingrid Michaelson;ZAYN,To Begin Again,To Begin Again,57,210826,False,0.438,0.3590,0,-9.734,1,0.0557,0.2100,0.000000,0.1170,0.120,76.332,4,[acoustic]
3,6lfxq3CG4xtTiEg7opyCyx,Kina Grannis,Crazy Rich Asians (Original Motion Picture Sou...,Can't Help Falling In Love,71,201933,False,0.266,0.0596,0,-18.515,1,0.0363,0.9050,0.000071,0.1320,0.143,181.740,3,[acoustic]
4,5vjLSffimiIP26QG5WcN2K,Chord Overstreet,Hold On,Hold On,82,198853,False,0.618,0.4430,2,-9.681,1,0.0526,0.4690,0.000000,0.0829,0.167,119.949,4,[acoustic]


# Genre Feature Engineering:

start by creating a list within track_genre where there are more than one genre

In [10]:
# concise aggregation: collect unique genres per track_id
genre_dataset = (
    data.groupby("track_id")["track_genre"]
        .agg(lambda vals: list({
            g.strip()
            for v in vals
            for g in (v if isinstance(v, (list, tuple)) else str(v).split(";"))
        }))
        .reset_index()
)

genre_dataset.head()

,track_id,track_genre
0,0000vdREvCVMxbQTkS888c,[german]
1,000Iz0K615UepwSJ5z2RE5,[minimal-techno]
2,000RDCYioLteXcutOjeweY,[hip-hop]
3,000qpdoc97IMTBvF8gwcpy,[minimal-techno]
4,0017XiMkqbTfF2AUOzlhj6,[comedy]


In [13]:
multi_genre_df = genre_dataset[genre_dataset["track_genre"].apply(len) > 1]

if multi_genre_df.empty:
    print("No multi-genre songs found.")
else:
    multi_genre_song = multi_genre_df.iloc[0]
    print(multi_genre_song.to_dict())


No multi-genre songs found.


Now changing the format of the genre values, from list to string

In [ ]:
import ast

# Assuming 'track_genre' column contains genre strings formatted as lists
# Remove the brackets and convert the string representation of lists to actual lists.
data["track_genre"] = data["track_genre"].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

# If you want to flatten and normalize the list to a single string (e.g., "forro") from lists
data["track_genre"] = data["track_genre"].apply(lambda x: x[0] if isinstance(x, list) and x else "uncategorized")

# Now, get unique genres
unique_genres = data["track_genre"].unique()
print(unique_genres)


<StringArray>
[   'acoustic',    'afrobeat',    'alt-rock', 'alternative',     'ambient',
       'anime', 'black-metal',   'bluegrass',       'blues',      'brazil',
 ...
     'spanish',       'study',     'swedish',   'synth-pop',       'tango',
      'techno',      'trance',    'trip-hop',     'turkish', 'world-music']
Length: 113, dtype: str


Now create a new aggregated genre category to support the genere indices:

- high-level genre category takes specific category names to make top-level genre category

In [30]:
# define the genre categories with dictionary
genre_categories = {
    'pop-mainstream': [
        'pop', 'pop-film', 'power-pop', 'k-pop', 'j-pop', 'mandopop', 
        'cantopop', 'indie-pop', 'synth-pop', 'j-idol'
    ],
    'rock': [
        'rock', 'alt-rock', 'grunge', 'punk', 'punk-rock', 'indie', 
        'psych-rock', 'garage', 'rock-n-roll', 'rockabilly', 'hard-rock'
    ],
    'electronic': [
        'house', 'techno', 'trance', 'dubstep', 'edm', 'electro', 'electronic',
        'drum-and-bass', 'deep-house', 'progressive-house', 'chicago-house',
        'detroit-techno', 'hardstyle', 'minimal-techno', 'idm'
    ],
    'hiphop-rnb': [
        'hip-hop', 'r-n-b'
    ],
    'metal': [
        'metal', 'heavy-metal', 'death-metal', 'black-metal', 'metalcore',
        'grindcore', 'hardcore'
    ],
    'country-folk': [
        'country', 'folk', 'honky-tonk', 'singer-songwriter', 'songwriter'
    ],
    'jazz-blues': [
        'jazz', 'blues', 'soul'
    ],
    'world-regional': [
        'latin', 'latino', 'afrobeat', 'brazil', 'forro', 'salsa', 'samba',
        'sertanejo', 'pagode', 'mpb', 'french', 'spanish', 'german', 'swedish',
        'indian', 'iranian', 'malay', 'turkish', 'j-dance', 'j-rock'
    ],
    'dance-club': [
        'dance', 'dancehall', 'disco', 'club', 'reggaeton', 'reggae', 'dub'
    ],
    'classical': [
        'classical', 'opera', 'new-age'
    ],
    'niche-mood': [
        'acoustic', 'ambient', 'anime', 'bluegrass', 'breakbeat', 'british',
        'children', 'chill', 'comedy', 'disney', 'emo', 'funk', 'gospel', 
        'goth', 'guitar', 'groove', 'happy', 'industrial', 'kids', 'party',
        'piano', 'romance', 'sad', 'show-tunes', 'ska', 'sleep', 'study', 'world-music'
    ]
}


Create function to add new top category to df

Call the function with the data df

In [35]:
def add_genre_cat(X: pd.DataFrame) -> pd.DataFrame:

    X = X.copy()
    genre_to_cat = {}

    for cat, genres in genre_categories.items():
            for genre in genres:
                genre_to_cat[genre] = cat

    X['genre_subcategory'] = X['track_genre'].map(genre_to_cat).fillna('uncategorized')
    print(X['genre_subcategory'].value_counts())
    
    return X

In [36]:
# Add the genre categories using the add_genre_cat function
multi_genre_df = add_genre_cat(data)

genre_subcategory
niche-mood        24134
world-regional    15528
electronic        10467
pop-mainstream     7585
rock               6308
metal              5516
dance-club         4340
country-folk       2781
classical          2549
uncategorized      2125
jazz-blues         1231
hiphop-rnb         1115
Name: count, dtype: int64


In [37]:
# Check for occurrences of "uncategorized" in the genre_subcategory column
uncategorized_count = multi_genre_df["genre_subcategory"].value_counts().get("uncategorized", 0)

print(f"Number of uncategorized genres: {uncategorized_count}")

# Optionally, show the rows with uncategorized genres if you want to inspect them
uncategorized_rows = multi_genre_df[multi_genre_df["genre_subcategory"] == "uncategorized"]
print(uncategorized_rows)

Number of uncategorized genres: 2125
                     track_id                        artists  \
2779   09gysnJpfQ3ublBmJDfcEC       Amaarae;Kali Uchis;Moliy   
2780   4fouWK6XVHhzl78KzQ1UjL                          GAYLE   
2781   07MDkzWARZaLEdKxo6yArG  THE ANXIETY;WILLOW;Tyler Cole   
2782   60a0Rd6pjrkxjPbaKzXjfq                    Linkin Park   
2783   2nLtzopw4rPReszdYBJU6h                    Linkin Park   
...                       ...                            ...   
81909  3TpcGANz2N705Bq4zc982H                   Roots Manuva   
81910  57qWtXga1hMwSfkhLDJCKQ        Everything But The Girl   
81911  0xcDUsknTawAv5VZKQ62aZ                     Wax Tailor   
81912  7LVOum3l1HrbitK9AoUPd7                     Cibo Matto   
81913  2Mt8qdFRaaIgzsf2bxwH4Y  Lovage;Nathaniel Merriweather   

                                              album_name  \
2779   SAD GIRLZ LUV MONEY Remix (feat. Kali Uchis an...   
2780                                             abcdefu   
2781          

In [38]:
# Get unique track_genre values that are categorized as "uncategorized"
uncategorized_tracks_unique = multi_genre_df[multi_genre_df["genre_subcategory"] == "uncategorized"]["track_genre"].unique()

# Create a DataFrame from the unique values
uncategorized_tracks_df = pd.DataFrame(uncategorized_tracks_unique, columns=["track_genre"])

# Specify the path to save the CSV file
output_file_path = "uncategorized_tracks.csv"  # Adjust the name/path as needed

# Write the DataFrame to a CSV file
uncategorized_tracks_df.to_csv(output_file_path, index=False)

print(f"Uncategorized tracks have been saved to {output_file_path}")


Uncategorized tracks have been saved to uncategorized_tracks.csv


Checking the csv we see that 'alternative, tango and trip hop' were not categorised, so we add them to the dictionary below

In [43]:
# define the genre categories with dictionary
genre_categories2 = {
    'pop-mainstream': [
        'pop', 'pop-film', 'power-pop', 'k-pop', 'j-pop', 'mandopop', 
        'cantopop', 'indie-pop', 'synth-pop', 'j-idol'
    ],
    'rock': [
        'rock', 'alt-rock', 'grunge', 'punk', 'punk-rock', 'indie', 
        'psych-rock', 'garage', 'rock-n-roll', 'rockabilly', 'hard-rock'
    ],
    'electronic': [
        'house', 'techno', 'trance', 'dubstep', 'edm', 'electro', 'electronic',
        'drum-and-bass', 'deep-house', 'progressive-house', 'chicago-house',
        'detroit-techno', 'hardstyle', 'minimal-techno', 'idm'
    ],
    'hiphop-rnb': [
        'hip-hop', 'r-n-b','trip-hop'
    ],
    'metal': [
        'metal', 'heavy-metal', 'death-metal', 'black-metal', 'metalcore',
        'grindcore', 'hardcore'
    ],
    'country-folk': [
        'country', 'folk', 'honky-tonk', 'singer-songwriter', 'songwriter'
    ],
    'jazz-blues': [
        'jazz', 'blues', 'soul'
    ],
    'world-regional': [
        'latin', 'latino', 'afrobeat', 'brazil', 'forro', 'salsa', 'samba',
        'sertanejo', 'pagode', 'mpb', 'french', 'spanish', 'german', 'swedish',
        'indian', 'iranian', 'malay', 'turkish', 'j-dance', 'j-rock', 'tango'
    ],
    'dance-club': [
        'dance', 'dancehall', 'disco', 'club', 'reggaeton', 'reggae', 'dub'
    ],
    'classical': [
        'classical', 'opera', 'new-age'
    ],
    'niche-mood': [
        'acoustic', 'alternative', 'ambient', 'anime', 'bluegrass', 'breakbeat', 'british',
        'children', 'chill', 'comedy', 'disney', 'emo', 'funk', 'gospel', 
        'goth', 'guitar', 'groove', 'happy', 'industrial', 'kids', 'party',
        'piano', 'romance', 'sad', 'show-tunes', 'ska', 'sleep', 'study', 'world-music'
    ]
}


In [44]:
def add_genre_cat(X: pd.DataFrame) -> pd.DataFrame:

    X = X.copy()
    genre_to_cat = {}

    for cat, genres in genre_categories2.items():
            for genre in genres:
                genre_to_cat[genre] = cat

    X['genre_subcategory'] = X['track_genre'].map(genre_to_cat).fillna('uncategorized')
    print(X['genre_subcategory'].value_counts())
    
    return X

In [45]:
multi_genre_df2 = add_genre_cat(data)

genre_subcategory
niche-mood        24383
world-regional    16522
electronic        10467
pop-mainstream     7585
rock               6308
metal              5516
dance-club         4340
country-folk       2781
classical          2549
hiphop-rnb         1997
jazz-blues         1231
Name: count, dtype: int64


There are no uncategorized in the genre sub-category now

In [49]:
multi_genre_df2

,track_id,artists,album_name,track_name,popularity,duration_ms,explicit,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,track_genre,genre_subcategory
0,5SuOikwiRyPMVoIQDJUgSV,Gen Hoshino,Comedy,Comedy,73,230666,False,0.676,0.4610,1,-6.746,0,0.1430,0.0322,0.000001,0.3580,0.7150,87.917,4,acoustic,niche-mood
1,4qPNDBW1i3p13qLCt0Ki3A,Ben Woodward,Ghost (Acoustic),Ghost - Acoustic,55,149610,False,0.420,0.1660,1,-17.235,1,0.0763,0.9240,0.000006,0.1010,0.2670,77.489,4,acoustic,niche-mood
2,1iJBSr7s7jYXzM8EGcbK5b,Ingrid Michaelson;ZAYN,To Begin Again,To Begin Again,57,210826,False,0.438,0.3590,0,-9.734,1,0.0557,0.2100,0.000000,0.1170,0.1200,76.332,4,acoustic,niche-mood
3,6lfxq3CG4xtTiEg7opyCyx,Kina Grannis,Crazy Rich Asians (Original Motion Picture Sou...,Can't Help Falling In Love,71,201933,False,0.266,0.0596,0,-18.515,1,0.0363,0.9050,0.000071,0.1320,0.1430,181.740,3,acoustic,niche-mood
4,5vjLSffimiIP26QG5WcN2K,Chord Overstreet,Hold On,Hold On,82,198853,False,0.618,0.4430,2,-9.681,1,0.0526,0.4690,0.000000,0.0829,0.1670,119.949,4,acoustic,niche-mood
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83674,2C3TZjDRiAzdyViavDJ217,Rainy Lullaby,#mindfulness - Soft Rain for Mindful Meditatio...,Sleep My Little Boy,21,384999,False,0.172,0.2350,5,-16.393,1,0.0422,0.6400,0.928000,0.0863,0.0339,125.995,5,world-music,niche-mood
83675,1hIz5L4IB9hN3WRYPOCGPw,Rainy Lullaby,#mindfulness - Soft Rain for Mindful Meditatio...,Water Into Light,22,385000,False,0.174,0.1170,0,-18.318,0,0.0401,0.9940,0.976000,0.1050,0.0350,85.239,4,world-music,niche-mood
83676,6x8ZfSoqDjuNa5SVP5QjvX,Cesária Evora,Best Of,Miss Perfumado,22,271466,False,0.629,0.3290,0,-10.895,0,0.0420,0.8670,0.000000,0.0839,0.7430,132.378,4,world-music,niche-mood
83677,2e6sXL2bYv4bSz6VTdnfLs,Michael W. Smith,Change Your World,Friends,41,283893,False,0.587,0.5060,7,-10.889,1,0.0297,0.3810,0.000000,0.2700,0.4130,135.960,4,world-music,niche-mood


This code:

- Collects unique genres from both track_genre and genre_subcategory.
- Creates a dictionary mapping each genre to a unique index.
- Constructs a genre_id column containing lists of indices based on the genre in each corresponding row.

In [53]:

# Step 1: Create a mapping for track_genre and genre_subcategory to numerical indices
# Use pd.concat to combine unique values from both columns
unique_genres = pd.concat([
    pd.Series(multi_genre_df2['track_genre'].unique()),
    pd.Series(multi_genre_df2['genre_subcategory'].unique())
]).unique()

# Create dictionary to map genres to numerical indices
genre_to_index = {genre: idx for idx, genre in enumerate(unique_genres)}

# Step 2: Create a new column 'genre_indices' by mapping and concatenating indices
multi_genre_df2['genre_id'] = multi_genre_df2.apply(
    lambda row: [genre_to_index[row['track_genre']], genre_to_index[row['genre_subcategory']]], axis=1
)

# Optionally, convert lists to string representation if needed
# multi_genre_df2['genre_indices'] = multi_genre_df2['genre_indices'].apply(lambda x: ', '.join(map(str, x)))

# Display the resulting DataFrame
print(multi_genre_df2[['track_genre', 'genre_subcategory', 'genre_id']])
multi_genre_df2


       track_genre genre_subcategory    genre_id
0         acoustic        niche-mood    [0, 113]
1         acoustic        niche-mood    [0, 113]
2         acoustic        niche-mood    [0, 113]
3         acoustic        niche-mood    [0, 113]
4         acoustic        niche-mood    [0, 113]
...            ...               ...         ...
83674  world-music        niche-mood  [112, 113]
83675  world-music        niche-mood  [112, 113]
83676  world-music        niche-mood  [112, 113]
83677  world-music        niche-mood  [112, 113]
83678  world-music        niche-mood  [112, 113]

[83679 rows x 3 columns]


,track_id,artists,album_name,track_name,popularity,duration_ms,explicit,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,track_genre,genre_subcategory,genre_id
0,5SuOikwiRyPMVoIQDJUgSV,Gen Hoshino,Comedy,Comedy,73,230666,False,0.676,0.4610,1,-6.746,0,0.1430,0.0322,0.000001,0.3580,0.7150,87.917,4,acoustic,niche-mood,"[0, 113]"
1,4qPNDBW1i3p13qLCt0Ki3A,Ben Woodward,Ghost (Acoustic),Ghost - Acoustic,55,149610,False,0.420,0.1660,1,-17.235,1,0.0763,0.9240,0.000006,0.1010,0.2670,77.489,4,acoustic,niche-mood,"[0, 113]"
2,1iJBSr7s7jYXzM8EGcbK5b,Ingrid Michaelson;ZAYN,To Begin Again,To Begin Again,57,210826,False,0.438,0.3590,0,-9.734,1,0.0557,0.2100,0.000000,0.1170,0.1200,76.332,4,acoustic,niche-mood,"[0, 113]"
3,6lfxq3CG4xtTiEg7opyCyx,Kina Grannis,Crazy Rich Asians (Original Motion Picture Sou...,Can't Help Falling In Love,71,201933,False,0.266,0.0596,0,-18.515,1,0.0363,0.9050,0.000071,0.1320,0.1430,181.740,3,acoustic,niche-mood,"[0, 113]"
4,5vjLSffimiIP26QG5WcN2K,Chord Overstreet,Hold On,Hold On,82,198853,False,0.618,0.4430,2,-9.681,1,0.0526,0.4690,0.000000,0.0829,0.1670,119.949,4,acoustic,niche-mood,"[0, 113]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83674,2C3TZjDRiAzdyViavDJ217,Rainy Lullaby,#mindfulness - Soft Rain for Mindful Meditatio...,Sleep My Little Boy,21,384999,False,0.172,0.2350,5,-16.393,1,0.0422,0.6400,0.928000,0.0863,0.0339,125.995,5,world-music,niche-mood,"[112, 113]"
83675,1hIz5L4IB9hN3WRYPOCGPw,Rainy Lullaby,#mindfulness - Soft Rain for Mindful Meditatio...,Water Into Light,22,385000,False,0.174,0.1170,0,-18.318,0,0.0401,0.9940,0.976000,0.1050,0.0350,85.239,4,world-music,niche-mood,"[112, 113]"
83676,6x8ZfSoqDjuNa5SVP5QjvX,Cesária Evora,Best Of,Miss Perfumado,22,271466,False,0.629,0.3290,0,-10.895,0,0.0420,0.8670,0.000000,0.0839,0.7430,132.378,4,world-music,niche-mood,"[112, 113]"
83677,2e6sXL2bYv4bSz6VTdnfLs,Michael W. Smith,Change Your World,Friends,41,283893,False,0.587,0.5060,7,-10.889,1,0.0297,0.3810,0.000000,0.2700,0.4130,135.960,4,world-music,niche-mood,"[112, 113]"
